In [1]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Graph

In [2]:
min_weight = 0
max_weight = 10
n_nodes = 5 * 10 ** 4
n_edges = 10 ** 5
# n_nodes, n_edges = 50000, 1000000 # <|-- takes 17.5 s to compute the maximum flow in a graph
source, destination = 0, 1 # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [3]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [4]:
np.random.uniform(min_weight, max_weight)

3.310166341801146

In [5]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

/tmp/ipykernel_1553/2680887831.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj_matrix = adjacency_matrix(Graph)


(50000, 50000)

In [6]:
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')
# total flow, flow first node
flow, flow_dict[1]

(2.635151862581475, {19322: 0, 2352: 0, 7853: 0, 1324: 0})

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [7]:
import pulp as pl

In [8]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

'cplex'

In [42]:
### TODO !!!!!!!!!!!!!!!!!
# Bcp de pertes de perf. Il faut le réimplémenter
# 
# 
# 
# 
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=100)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [43]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=3))

3

## Master

In [51]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)

paths = get_n_paths (Graph, source, destination, max_iter = 10)

lpProb += 0 # initiate the objective function to 0 (necessary line of code)

# pour chaque path ajouter une variable et ses coef. dans les contraintes
for index_path, path in enumerate (paths) :

    # add a var path
    prov = pl.LpVariable("X"+str(index_path), 0, None, pl.LpContinuous)
    lpProb.objective.addterm(prov, 1)
    
    # for each edge on the path, search the associated constraint (indexed by edge)
    for index in range(len(path)-1) :
        index_ctr = str((path[index], path[index + 1]))
        if index_ctr in lpProb.constraints:
            lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
        else:
            lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]

solver = pl.CPLEX_PY()

result = lpProb.solve(solver)

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 207 rows and 10 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.17 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.17 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.17 ticks)
Cplex status= 101


In [52]:
# print results
for v in lpProb.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

X0 = 0.0244774868389519


In [53]:
lpProb

path_generation:
MAXIMIZE
1*X0 + 1*X1 + 1*X2 + 1*X3 + 1*X4 + 1*X5 + 1*X6 + 1*X7 + 1*X8 + 1*X9 + 0
SUBJECT TO
(0, 10560): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 <= 5.7468572665

(10560, 35959): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 2.4289313779

(35959, 31458): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 5.59320958426

(31458, 9141): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 1.85371806728

(9141, 1974): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 <= 4.37156595069

(1974, 34317): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 6.14779449386

(34317, 24479): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 0.869368788127

(24479, 15699): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 1.24215897449

(15699, 1981): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 5.58991346638

(1981, 49312): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 3.56069166066

(49312, 45477): X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9
 <= 7.1455

## Dual variables

In [54]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

In [55]:
get_duals(lpProb)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

## Plus court chemin

## Coût réduit